In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_recall_curve, average_precision_score, roc_auc_score, confusion_matrix
from sklearn.feature_selection import mutual_info_classif

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

from statsmodels.tsa.stattools import grangercausalitytests

from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
seed = 20201106 # the deadline date

In [3]:
# Columns used in the Random Forests model from Muchlinski et al. (2016)

rf_cols = ["ager", "agexp", "anoc", "army85", 
           "autch98", "auto4", "autonomy", "avgnabo",
           "centpol3", "coldwar", "decade1", "decade2", 
           "decade3", "decade4", "dem", "dem4",
           "demch98", "dlang", "drace", "drel", 
           "durable", "ef", "ef2", "ehet", 
           "elfo", "elfo2", "etdo4590", "expgdp", 
           "exrec", "fedpol3", "fuelexp", "gdpgrowth", 
           "geo1", "geo2", "geo34", "geo57",
           "geo69", "geo8", "illiteracy", "incumb", 
           "infant", "inst", "inst3", "life",
           "lmtnest", "major", "manuexp", "milper",
           "mirps0", "mirps1", "mirps2", "mirps3", 
           "nat_war", "ncontig", "nmgdp", "nmdp4_alt",
           "numlang", "nwstate", "oil", "p4mchg", 
           "parcomp", "parreg", "part", "partfree", 
           "plural", "plurrel", "pol4", "pol4m", 
           "pol4sq", "polch98", "polcomp", "popdense", 
           "presi", "pri", "proxregc", "reg", 
           "regd4_alt", "relfrac", "seceduc", "second",
           "semipol3", "sip2", "sxpnew", "sxpsq",
           "tnatwar", "trade", "warhist", "xconst"]

In [4]:
# Load raw data 
df = pd.read_csv('data/SambnisImp.csv', usecols=rf_cols+['cowcode', 'warstds', 'year'])

# Print dimension information about the dataset
print(f'Raw data dimensions: {df.shape}')

# Print first 5 columns
df.head()

Raw data dimensions: (7140, 91)


,cowcode,year,warstds,autonomy,popdense,army85,milper,trade,nmgdp,autch98,...,mirps1,mirps2,mirps3,sxpsq,pol4sq,decade1,decade2,decade3,decade4,proxregc
0,700,1945,0,0.005151,118.554791,129472.9042,121.087366,72.881375,4508.131692,0,...,0.16935,0.313143,0.373714,0.052989,61.493014,0,0,0,0,0.143299
1,700,1946,0,0.000000,117.756342,129413.0225,121.885359,72.900089,4491.340308,0,...,0.00000,1.000000,0.000000,0.052663,100.000000,0,0,0,0,1.000000
2,700,1947,0,0.000000,118.280656,130431.0145,122.780608,72.962880,4484.267686,0,...,0.00000,1.000000,0.000000,0.052891,100.000000,0,0,0,0,1.000000
3,700,1948,0,0.000000,118.325869,126781.6866,118.256427,73.102449,4474.082672,0,...,0.00000,1.000000,0.000000,0.052902,100.000000,0,0,0,0,1.000000
4,700,1949,0,0.000000,118.312296,130979.2470,122.245074,72.850389,4497.299503,0,...,0.00000,1.000000,0.000000,0.052706,100.000000,0,0,0,0,1.000000


In [5]:
def impute_missing_geographical_data(df, year=2000):
    # For each country in the year 2000
    # Replace zeros in the geographical areas with the data from year 1999
    
    geos = ['geo1', 'geo2', 'geo34', 'geo57', 'geo69', 'geo8']
    
    for country in df.cowcode.unique():
        idx = df[(df.cowcode == country) & (df.year == year)].index
        if idx.empty:
            continue
        
        #for geo in geos:
        df.loc[idx,geos] = df.loc[idx-1,geos].values
    
    return df


def find_categorical_features(df):
    cat_cols = np.zeros(df.shape[1])

    for i, col in enumerate(df):
        if len(df[col].unique()) <= 5:
            cat_cols[i] = 1
    
    return cat_cols

df = impute_missing_geographical_data(df)

In [6]:
df[rf_cols] = MinMaxScaler().fit_transform(df[rf_cols].values)

In [7]:
country_groups = df.groupby(by='cowcode')

country_dfs = [country_groups.get_group(x) for x in country_groups.groups]

cw_country_dfs = [x for x in country_dfs if x.warstds.sum() > 0]

In [51]:
y = []
X = []

N = 10

for cwcd in country_dfs:
    #print(cwcd.shape)
    cwcd = cwcd.sort_values(by='year', axis=0)
    
    #print(cwcd.shape)
    
    cwcd_Y = cwcd.loc[:,'warstds'] 
    cwcd = cwcd.drop(['cowcode', 'year', 'warstds'], axis=1)
    
    if cwcd.shape[0] <= (N*2):
        continue
    
    cwcd_X_windowed = np.zeros((cwcd.shape[0]-N, N, cwcd.shape[1]))
    cwcd_Y_windowed = np.zeros((cwcd.shape[0]-N,))
    
    for i in range(cwcd.shape[0]-N):
        cwcd_X_windowed[i,:,:] = cwcd.iloc[i:i+N,:].values # Get data from previous years
        cwcd_Y_windowed[i] = cwcd_Y.iloc[i+N] # Append the label for the next year
    
    X.append(cwcd_X_windowed)
    y.append(cwcd_Y_windowed)

X = np.concatenate(X)
y = np.concatenate(y)

In [52]:
X.shape, y.shape

((5351, 10, 88), (5351,))

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed, stratify=y)

In [54]:
# Function to create model, required for KerasClassifier
def create_model(LSTM_neuron = 10, drop_rate=0.2):
    # create model
    model = Sequential()
    model.add(LSTM(LSTM_neuron, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(drop_rate))
    model.add(Dense(1, activation='sigmoid'))
    #compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
np.random.seed(seed)

In [55]:
"""LSTM = Pipeline([('smpl', RandomUnderSampler()),
               ('lstm', keras_estimator)])

param_grid = {'lstm__LSTM_neuron': [25, 100],
              'lstm__epochs': [10],
             'lstm__drop_rate': [0.2, 0.6],
             'lstm__batch_size': [8, 50],
              'smpl__sampling_strategy': [0.125, 0.25, 0.5]
              }"""

# Function to create model, required for KerasClassifier
def create_model(LSTM_neuron=10, n_LSTM_L=1, drop_rate=0.2):
    # create model
    model = Sequential()
    
    if n_LSTM_L-1 > 0:
        model.add(LSTM(LSTM_neuron, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
        model.add(Dropout(drop_rate))
    else:
        model.add(LSTM(LSTM_neuron, input_shape=(X.shape[1], X.shape[2])))
        model.add(Dropout(drop_rate))
         
    for i in range(n_LSTM_L-2):
        model.add(LSTM(LSTM_neuron, return_sequences=True))
        model.add(Dropout(drop_rate))
        
    if n_LSTM_L-1 > 0:   
        model.add(LSTM(LSTM_neuron))
        model.add(Dropout(drop_rate))

    model.add(Dense(1, activation='sigmoid'))
    #compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
np.random.seed(seed)
    
param_grid = {'LSTM_neuron': [5, 50],
              'n_LSTM_L': [2, 4],
              'epochs': [20],
             'drop_rate': [0.2, 0.6],
             'batch_size': [5, 10]
             }

class_weights = compute_class_weight('balanced', np.unique(y), y)
class_weights[1] = class_weights[1]/3
class_weights = dict(enumerate(class_weights))

keras_estimator = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
grid = GridSearchCV(estimator=keras_estimator, param_grid=param_grid, n_jobs=-1, cv=3, scoring='average_precision')
grid_result = grid.fit(X_train, y_train, class_weight=class_weights)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

c:\users\victo\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass classes=[0. 1.], y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Epoch 1/20
428/428 [==============================] - 8s 20ms/step - loss: 0.3989 - accuracy: 0.9841 
Epoch 2/20
428/428 [==============================] - 8s 19ms/step - loss: 0.3687 - accuracy: 0.9864
Epoch 3/20
428/428 [==============================] - 9s 20ms/step - loss: 0.3815 - accuracy: 0.9864
Epoch 4/20
428/428 [==============================] - 8s 19ms/step - loss: 0.3604 - accuracy: 0.9862
Epoch 5/20
428/428 [==============================] - 8s 20ms/step - loss: 0.3654 - accuracy: 0.9864
Epoch 6/20
428/428 [==============================] - 7s 17ms/step - loss: 0.3423 - accuracy: 0.9853
Epoch 7/20
428/428 [==============================] - 8s 19ms/step - loss: 0.3514 - accuracy: 0.9848
Epoch 8/20
428/428 [==============================] - 9s 20ms/step - loss: 0.3387 - accuracy: 0.9766
Epoch 9/20
428/428 [==============================] - 9s 20ms/step - loss: 0.3550 - accuracy: 0.9843
Epoch 10/20
428/428 [==============================] - 9s 20ms/step - loss: 0.3259 - accur

In [64]:
class_weights = compute_class_weight('balanced', np.unique(y), y)
class_weights[1] = class_weights[1]/2
class_weights = dict(enumerate(class_weights))

best_params = grid_result.best_params_
model = create_model(best_params["LSTM_neuron"], best_params["n_LSTM_L"], best_params["drop_rate"])
history = model.fit(X_train, y_train, epochs=50, batch_size=best_params["batch_size"], validation_data=(X_test, y_test), verbose=2, shuffle=False, class_weight=class_weights)


Epoch 1/50
428/428 - 13s - loss: 0.5139 - accuracy: 0.9348 - val_loss: 0.5296 - val_accuracy: 0.9860
Epoch 2/50
428/428 - 9s - loss: 0.4758 - accuracy: 0.9636 - val_loss: 0.4990 - val_accuracy: 0.9860
Epoch 3/50
428/428 - 9s - loss: 0.4676 - accuracy: 0.9329 - val_loss: 0.4673 - val_accuracy: 0.9860
Epoch 4/50
428/428 - 9s - loss: 0.4537 - accuracy: 0.9290 - val_loss: 0.3469 - val_accuracy: 0.9795
Epoch 5/50
428/428 - 9s - loss: 0.4688 - accuracy: 0.9451 - val_loss: 0.4409 - val_accuracy: 0.9599
Epoch 6/50
428/428 - 8s - loss: 0.4201 - accuracy: 0.9196 - val_loss: 0.3681 - val_accuracy: 0.9542
Epoch 7/50
428/428 - 9s - loss: 0.4009 - accuracy: 0.8909 - val_loss: 0.3725 - val_accuracy: 0.7180
Epoch 8/50
428/428 - 8s - loss: 0.4029 - accuracy: 0.8598 - val_loss: 0.3805 - val_accuracy: 0.7311
Epoch 9/50
428/428 - 8s - loss: 0.3599 - accuracy: 0.8773 - val_loss: 0.2091 - val_accuracy: 0.8534
Epoch 10/50
428/428 - 8s - loss: 0.4106 - accuracy: 0.8124 - val_loss: 0.2615 - val_accuracy: 0.820

In [65]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_46 (LSTM)               (None, 10, 5)             1880      
_________________________________________________________________
dropout_46 (Dropout)         (None, 10, 5)             0         
_________________________________________________________________
lstm_47 (LSTM)               (None, 10, 5)             220       
_________________________________________________________________
dropout_47 (Dropout)         (None, 10, 5)             0         
_________________________________________________________________
lstm_48 (LSTM)               (None, 10, 5)             220       
_________________________________________________________________
dropout_48 (Dropout)         (None, 10, 5)             0         
_________________________________________________________________
lstm_49 (LSTM)               (None, 5)               

In [66]:
y_pred = model.predict_classes(X_test)
print(np.unique(y_pred))
confusion_matrix(y_test, y_pred)

[0 1]


array([[835, 221],
       [ 11,   4]], dtype=int64)

In [67]:
y_pred = model.predict_classes(X_train)
print(np.unique(y_pred))
confusion_matrix(y_train, y_pred)

[0 1]


array([[3357,  865],
       [   3,   55]], dtype=int64)